In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor

## Импорт Датасетов

In [11]:
training_data = datasets.FashionMNIST(
    root = "data", 
    train = True,
    download = True,
    transform = ToTensor(),
)

testing_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

In [12]:
batch_size = 64

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size = batch_size)
test_dataloader = torch.utils.data.DataLoader(testing_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"X_test shape: {X.shape} | y_test shape: {y.shape}")
    break

X_test shape: torch.Size([64, 1, 28, 28]) | y_test shape: torch.Size([64])


## Классическая нейронная сеть (Полносвязные слои с функцией активации ReLU)

Для обучения используется SGD

In [13]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"

class NeuralNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = torch.nn.Flatten()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(28*28, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 10)
        )

    def forward(self, X):
        X = self.flatten(X)
        logits = self.linear_relu_stack(X)

        return logits

nn_model = NeuralNet().to(device)

print(nn_model)        

NeuralNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(nn_model.parameters(), lr = 1e-3)

def train(dataloader, model, loss_func, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        preds = model(X)
        loss = loss_func(preds, y)

        #backprop
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Loss: {loss} | {current} / {size}")

def test(dataloader, model, loss_func):
    size = len(dataloader.dataset)
    batches_amount = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            preds = model(X)
            test_loss += loss_func(preds, y).item()
            correct += (preds.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= batches_amount
    correct /= size

    print(f"Test || Loss: {test_loss:>5f} || Accuracy: {correct*100:>.2f}%")


epochs = 8
for e in range(epochs):
    print(f"Epoch {e+1} \n ===============================================")
    train(train_dataloader, nn_model, loss_func, optimizer)
    test(test_dataloader, nn_model, loss_func)

Epoch 1 
Loss: 2.310110569000244 | 64 / 60000
Loss: 2.293985366821289 | 6464 / 60000
Loss: 2.275315046310425 | 12864 / 60000
Loss: 2.262678623199463 | 19264 / 60000
Loss: 2.242948293685913 | 25664 / 60000
Loss: 2.227524995803833 | 32064 / 60000
Loss: 2.2212908267974854 | 38464 / 60000
Loss: 2.195888042449951 | 44864 / 60000
Loss: 2.184767246246338 | 51264 / 60000
Loss: 2.1555087566375732 | 57664 / 60000
Test || Loss: 2.152703 || Accuracy: 58.46%
Epoch 2 
Loss: 2.1625683307647705 | 64 / 60000
Loss: 2.146975517272949 | 6464 / 60000
Loss: 2.0913848876953125 | 12864 / 60000
Loss: 2.1056289672851562 | 19264 / 60000
Loss: 2.052797794342041 | 25664 / 60000
Loss: 2.0085554122924805 | 32064 / 60000
Loss: 2.0213403701782227 | 38464 / 60000
Loss: 1.9461053609848022 | 44864 / 60000
Loss: 1.9373857975006104 | 51264 / 60000
Loss: 1.8832042217254639 | 57664 / 60000
Test || Loss: 1.874549 || Accuracy: 60.83%
Epoch 3 
Loss: 1.9010343551635742 | 64 / 60000
Loss: 1.8666703701019287 | 6464 / 60000
Loss: 1

## Свёрточная нейронная сеть (+ использование нормирования)

Для обучения использовалась Adam

In [16]:
import torchvision

transform = torchvision.transforms.Compose(
    [ToTensor(),
    torchvision.transforms.Normalize((.5), (.5))]
)

train_set = torchvision.datasets.FashionMNIST(
    root = "data_cnn",
    train = True,
    download = True,
    transform = transform,
)

test_set = torchvision.datasets.FashionMNIST(
    root = "data_cnn",
    train = False,
    download = True,
    transform = transform,
)

100.0%
100.0%
100.0%
100.0%


In [17]:
batch_size = 64

train_dataloader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"X_test.shape: {X.shape} | y_test.shape: {y.shape}")
    break

X_test.shape: torch.Size([64, 1, 28, 28]) | y_test.shape: torch.Size([64])


In [36]:
class ConvolutionalNet(torch.nn.Module):
    def __init__(self):
        super(ConvolutionalNet, self).__init__()
        
        self.first_layer = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, padding = 1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2),
        )

        self.second_layer = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 0),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2),
        )

        self.fc1 = torch.nn.Linear(in_features = 64*6*6, out_features = 600)
        self.drop = torch.nn.Dropout2d(.25)
        self.fc2 = torch.nn.Linear(in_features = 600, out_features = 120)
        self.fc3 = torch.nn.Linear(in_features = 120, out_features = 10)


    def forward(self, X):
        out_sequential1 = self.first_layer(X)
        out_sequential2 = self.second_layer(out_sequential1)
        out_sequential2 = out_sequential2.view(out_sequential2.size(0), -1)
        out_fc1 = self.fc1(out_sequential2)
        out_drop = self.drop(out_fc1)
        out_fc2 = self.fc2(out_drop)
        out_fc3 = self.fc3(out_fc2)

        return out_fc3

## Переопределим некоторые уже известные переменные для удобства

In [37]:
cnn_model = ConvolutionalNet().to(device)
print(cnn_model)

ConvolutionalNet(
  (first_layer): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (second_layer): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


In [38]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn_model.parameters(), lr = 1e-3)

epochs = 5
for e in range(epochs):
    print(f"Epoch {e+1} \n ===================================")
    train(train_dataloader, cnn_model, loss_func, optimizer)
    test(test_dataloader, cnn_model, loss_func)

Epoch 1 
Loss: 2.295938491821289 | 64 / 60000
Loss: 0.5700045824050903 | 6464 / 60000
Loss: 0.2895480692386627 | 12864 / 60000
Loss: 0.5124019384384155 | 19264 / 60000
Loss: 0.4573103189468384 | 25664 / 60000
Loss: 0.47280430793762207 | 32064 / 60000
Loss: 0.24157072603702545 | 38464 / 60000
Loss: 0.6042357087135315 | 44864 / 60000
Loss: 0.3585544228553772 | 51264 / 60000
Loss: 0.2593194544315338 | 57664 / 60000
Test || Loss: 0.387314 || Accuracy: 85.69%
Epoch 2 
Loss: 0.2269202470779419 | 64 / 60000
Loss: 0.3157084882259369 | 6464 / 60000
Loss: 0.21053919196128845 | 12864 / 60000
Loss: 0.38154926896095276 | 19264 / 60000
Loss: 0.49371281266212463 | 25664 / 60000
Loss: 0.4264756143093109 | 32064 / 60000
Loss: 0.21364636719226837 | 38464 / 60000
Loss: 0.46278896927833557 | 44864 / 60000
Loss: 0.2566431164741516 | 51264 / 60000
Loss: 0.2287500500679016 | 57664 / 60000
Test || Loss: 0.320918 || Accuracy: 88.28%
Epoch 3 
Loss: 0.18783168494701385 | 64 / 60000
Loss: 0.3981507420539856 | 646